In [1]:
import cdsapi
import xarray as xr
import pandas as pd
import os
import netCDF4
import h5netcdf


ModuleNotFoundError: No module named 'cdsapi'

/opt/python-env/bin:/home/siruwww/.vscode-server/bin/b58957e67ee1e712cebf466b995adf4c5307b2bd/bin/remote-cli:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files/dotnet/:/mnt/c/Program Files/Microsoft SQL Server/150/Tools/Binn/:/mnt/c/Program Files (x86)/Windows Kits/10/Windows Performance Toolkit/:/mnt/c/Users/Administrator/AppData/Local/Programs/Quarto/bin:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0/:/mnt/c/Windows/System32/OpenSSH/:/mnt/c/Program Files/dotnet/:/mnt/d/Scripts/:/mnt/d/:/mnt/c/Users/Administrator/AppData/Local/Microsoft/WindowsApps:/mnt/c/Users/Administrator/AppData/Local/Programs/Microsoft VS Code/bin:/snap/bin

In [1]:
# 创建客户端
client = cdsapi.Client()

# 数据集名称
dataset = "reanalysis-era5-land"

# 请求的变量
variables = [
    "total_precipitation",          # 降水
]

# 北美范围
north_america_extent = {
    "area": [85, 190, 10, 310],  # 格式：[北纬, 西经, 南纬, 东经]
}

# 时间分辨率（6小时）
time_intervals = [
    "00:00", "12:00"
]

# 下载 2024 年 1 到 11 月的数据
for month in range(1, 12):
    print(f"Downloading data for month {month:02d}...")

    # 请求参数
    request = {
        "format": "netcdf",
        "product_type": "reanalysis",
        "variable": variables,
        "year": "2024",
        "month": f"{month:02d}",
        "day": [f"{day:02d}" for day in range(1, 32)],  # 天数
        "time": time_intervals,
        **north_america_extent,  # 添加地理范围
    }

    # 输出文件路径
    output_filename = f"month_data/era5_2024_month_{month:02d}_north_america.nc"

    # 执行抓取
    client.retrieve(dataset, request, output_filename)
    print(f"Data for month {month:02d} saved to {output_filename}.")

NameError: name 'cdsapi' is not defined

In [1]:
import xarray as xr
import os

# 文件路径
data_directory = "month_data"
file_paths = sorted([os.path.join(data_directory, f) for f in os.listdir(data_directory) if f.endswith(".nc")])

# 使用 dask 进行分块加载
datasets = [xr.open_dataset(file_path, engine="netcdf4", chunks={"latitude": 100, "longitude": 100}) for file_path in file_paths]

# 沿时间维度 (valid_time) 拼接
combined_ds = xr.concat(datasets, dim="valid_time")

# 检查合并结果
print(combined_ds)

# 保存合并结果到 NetCDF 文件（启用压缩）
output_file = "combined_precipitation_data_2024.nc"
combined_ds.to_netcdf(output_file, encoding={"tp": {"zlib": True, "complevel": 5}})
print(f"Combined dataset saved to {output_file}")

/tmp/ipykernel_1985024/3066542303.py:9: UserWarning: The specified chunks separate the stored chunks along dimension "latitude" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  datasets = [xr.open_dataset(file_path, engine="netcdf4", chunks={"latitude": 100, "longitude": 100}) for file_path in file_paths]
/tmp/ipykernel_1985024/3066542303.py:9: UserWarning: The specified chunks separate the stored chunks along dimension "longitude" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  datasets = [xr.open_dataset(file_path, engine="netcdf4", chunks={"latitude": 100, "longitude": 100}) for file_path in file_paths]
/tmp/ipykernel_1985024/3066542303.py:9: UserWarning: The specified chunks separate the stored chunks along dimension "latitude" starting at index 100. This could degrade performance. Instead, consider rechunking after loading.
  datasets = [xr.open_dataset(file_path, engine="netcd

<xarray.Dataset> Size: 5GB
Dimensions:     (valid_time: 1301, latitude: 751, longitude: 1201)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 10kB 2024-01-01 ... 2024-11-21
  * latitude    (latitude) float64 6kB 85.0 84.9 84.8 84.7 ... 10.2 10.1 10.0
  * longitude   (longitude) float64 10kB 190.0 190.1 190.2 ... 309.8 309.9 310.0
    expver      (valid_time) <U4 21kB dask.array<chunksize=(124,), meta=np.ndarray>
Data variables:
    tp          (valid_time, latitude, longitude) float32 5GB dask.array<chunksize=(31, 100, 100), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-26T05:23 GRIB to CDM+CF via cfgrib-0.9.1...
Combined dataset saved to combined_precipitation_data_2024.nc


In [2]:
import xarray as xr

file_path = "combined_precipitation_data_2024.nc"
ds = xr.open_dataset(file_path, engine="netcdf4")

# 打印数据集的结构
print(ds)


<xarray.Dataset> Size: 5GB
Dimensions:     (valid_time: 1301, latitude: 751, longitude: 1201)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 10kB 2024-01-01 ... 2024-11-21
  * latitude    (latitude) float64 6kB 85.0 84.9 84.8 84.7 ... 10.2 10.1 10.0
  * longitude   (longitude) float64 10kB 190.0 190.1 190.2 ... 309.8 309.9 310.0
    expver      (valid_time) <U4 21kB ...
Data variables:
    tp          (valid_time, latitude, longitude) float32 5GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-26T05:23 GRIB to CDM+CF via cfgrib-0.9.1...
